In [1]:
print("Data_Ingestion")

Data_Ingestion


In [6]:
%pwd

'g:\\RealProject\\End_to_End\\research'

In [ ]:
import os
os.chdir("../")
%pwd

In [9]:
from src.datascience import logger

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir: Path 
    source_URL: str 
    local_data_file: Path
    unzip_dir: Path

In [11]:
from src.datascience.constant import *
from src.datascience.utils.common import read_yaml, create_directories


In [ ]:

class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH): 
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config




In [17]:
# component - Data Ingestion
from urllib import request
import os
from src.datascience import logger
import zipfile


In [18]:
class DataIngestion:
    def __init__(self,config:DataIngestionconfig) -> None:
        self.config = config
    
    # Downloading the Zip file
    def download_file(self) -> None:
        """
        Download the Zip file
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else: 
            logger.info(f"{filename} already exists")

    def extract_zip_file(self) -> None:
        """
        zip_file_path: str
        Extracts the zip file into the data directoty
        function return None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)  


    

In [20]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2025-06-29 03:32:46,971: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-29 03:32:46,973: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-29 03:32:46,976: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-29 03:32:46,978: INFO: common: created directory at: artifact]
[2025-06-29 03:32:46,980: INFO: common: created directory at: artifact/data_ingestion]
[2025-06-29 03:32:47,687: INFO: 531898317: artifact/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 390E:26132D:F4C86:2D1689:686065DE
Accept-Ranges: bytes
Date: Sat, 